# Predictions With Our Fine-Tuned YOLOv11 Model


#### Course: Deep Neural Engineering (IM1102)
#### Group: Ellen Cordemans, Ilse Harmers & Sem Pepels

The code in this notebook is adapted from [1].


---



**References**

[1] Kondrackis, L. (2024, December 5). How to Train YOLOv11 Instance
Segmentation on a Custom Dataset. Roboflow Blog. Retrieved April 15, 2025, from https://blog.roboflow.com/train-yolov11-instance-segmentation/

## Preparations

In [ ]:
# Checking the availability of Google Colab's GPU.
!nvidia-smi

In [ ]:
# Installing Roboflow and Ultralytics libraries.
!pip install -q "roboflow==1.1.58" "ultralytics==8.3.95"

In [ ]:
# Importing important models, functions and libraries.
from google.colab import userdata
from roboflow import Roboflow
import torch
from ultralytics import YOLO
import os
import re
from IPython.display import display
from PIL import Image
import locale

In [ ]:
# Downloading our dataset from Roboflow.
ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("workspace-v0o55").project("parsing-house-facades")
version = project.version(10)      # Train-Val-Test: 70-15-15.
dataset = version.download("yolov11")
dataset_path = f"{dataset.location}/data.yaml"

## Evaluate Previously Fine-Tuned Model

In [ ]:
# Loading in our best model from GitHub.
locale.getpreferredencoding = lambda: "UTF-8"
!git clone https://github.com/Jow-Dit-Is-Sem/Parsing-House-Facades.git
model = YOLO("/content/Parsing-House-Facades/Fine-tuned_models/best.pt")

In [ ]:
# Checking whether cuda is available for Torch to use as device.
if torch.cuda.is_available():
    print("Cuda is available. Torch will use Cuda.")
    device = "cuda"
elif torch.backends.mps.is_available():
    print("MPS is available. Torch will use MPS.")
    device = "mps"
else:
    print("GPU is not available. Torch will fall back to CPU.")
    device = "cpu"

In [ ]:
# Determining metrics on the test set.
metrics = model.val(data=dataset_path, split="test")

In [ ]:
# In this cell, we display the quantitative results on the test set (as a check).
# Printing class labels.
print(f"Labels: {metrics.names}")

# Box AP metrics.
AP50b = metrics.box.ap50
AP50_95b = metrics.box.ap
# Segmentation AP metrics.
AP50s = metrics.seg.ap50
AP50_95s = metrics.seg.ap

print(f"AP@50: {AP50b} (B) & {AP50s} (S)")
print(f"AP@50-95: {AP50_95b} (B) & {AP50_95s} (S)\n")

# Box mAP metrics.
mAP50b = metrics.box.map50
mAP50_95b = metrics.box.map
# Segmentation mAP metrics.
mAP50s = metrics.seg.map50
mAP50_95s = metrics.seg.map

print(f"mAP@50: {mAP50b} (B) & {mAP50s} (S)")
print(f"mAP@50-95: {mAP50_95b} (B) & {mAP50_95s} (S)")

In [ ]:
# Displaying the confusion matrix of the test set for IoU = 0.45 (default value in Ultralytics library).
folder_segment_content = os.listdir("/content/runs/segment")
test_folder = None

# Extracting the most recently saved 'val' directory in which the test results were stored.
def extract_last_number(s):
    match = re.search(r'val(\d+)', s)
    if match:
        return int(match.group(1))
    elif s == "val":
        return -float('inf')
    return float('inf')

filtered_list = [s for s in folder_segment_content if re.search(r'val\d*', s)]
sorted_list = sorted(filtered_list, key=extract_last_number, reverse=True)
test_folder = sorted_list[0]

# Displaying the (normalized) confusion matrix of the test data.
img = Image.open(f"/content/runs/segment/{test_folder}/confusion_matrix_normalized.png")
img_resized = img.resize((img.width // 4, img.height // 4))
display(img_resized)

## Visual Predictions With Our Model

In [ ]:
# Saving the annotated predictions on the test images for the visual comparisons mentioned in the report.
# We set 'iou=0.6' such that we have the same threshold in the val and predict modes.
model.predict(source=f"{dataset.location}/test/images", iou=0.6, save=True)

In [ ]:
# Saving the prediction results to a ZIP file.
# This line might become deprecated when running multiple runs in one session, since the new directory might be saved as 'predict{+1}'.
!zip -r /content/runs/segment/predict/test_images.zip /content/runs/segment/predict